# 트리의 앙상블

#### 정형 데이터와 비정형 데이터
csv와 같은 데이터를 정형 데이터라고 부름. 어떤 구조로 되어있다는 뜻.  
반대되는 데이터는 비정형 데이터라고 함. 텍스트 데이터, 디카로 찍은 사진, 핸드폰으로 듣는 디지털 음악 등이 있다.

정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘이 **앙상블 학습**이다.  
대부분 결정 트리를 기반으로 만들어져 있음.  
비정형 데이터에는 신경망 알고리즘이 좋다. 전통적인 머신러닝 방법으로는 모델을 만들기 까다롭다.  

## 랜덤포레스트
결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만듬. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듬.  
랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 만드는 것이 좀 독특함.  
입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만듬. 이때 한 샘플이 중복되어 추출될 수도 있음.  
  
**부트스트랩 샘플** : 1000개 샘플이 들어있는 가방에서 100의 샘플을 뽑는다면 먼저 1개 뽑고, 다시 가방에 넣고 이런식으로 100번 뽑아 만든 샘플.  
기본적으로 부트스트랩 샘플은 훈련 세트의 크기와 같게 만듬.  
  
또 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음.  
분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수이 제곱근만큼의 특성을 선택함.  
회귀 모델인 RandomForestRegressor는 전체 특성을 사용함.  
  
사이킷런의 랜덤포레스트는 기본적으로 100개의 결정 트리를 훈련.  
분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음.  
회귀일 때는 단순히 각 트리의 예측을 평균.  
  
랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있고 종종 기본 매개변수 설정만으로도 아주 좋은 결과를 냄.  
  
RandomForestClassifier 클래스를 화이트 와인을 분류하는 문제에 적용.  
우선 와인 데이터 셋을 판다스로 불러오고 훈련 세트와 테스트 세트로 나눔.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

cross_validate() 함수를 사용해 쇼차 검증 수행.  
이 클래스는 사용하는 결정 트리의 기본값이 100개이므로 n_jobs 매개변수를 -1로 지정.  
또 return_train_score 매개변수를 True로 지정(기본값은 False)하여 검증 점수뿐 아니라 훈련 세트에 대한 점수도 같이 반환.  
과대 적합을 파악하는 데 용이.

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


출력된 결과는 다소 과대적합.  
랜덤 포레스트는 결정 트리의 앙상블이라서 DecisionTreeClassifier가 제공하는 매개변수를 모두 제공.  
criterion, max_depth, max_features, in_smaples_split, min_impurity_decrease, min_samples_leaf 등  
또, 결정 트리의 큰 장점 중 하나인 특성 중요도를 계산한다.  
랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것이다.  
특성 중요도를 출력하면 다음과 같다.

In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


앞 절 결정 트리에서 만든 특성 중요도와 비교하면 두 번째 특성인 당도의 중요도가 감소하고 알코올 도수와 pH 특성의 중요도가 조금 상승.  
이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 틀를 훈련하기 때문이다.  
하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는 셈인데,  
이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 된다. 

RandomForestClassifier에는 자체적으로 모델을 평가하는 점수를 얻을 수 있는 기능이 있다.  
훈련 세트에서 중복을 허용하여 부트스트랩 샘플을 만들어 결정 트리를 훈련하는데, 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있다.  
이런 샘플을 OOB 샘플이라고 하고, 이 남는 샘플을 사용하여 부트슽랩 샘플로 훈련한 결정 트리를 평가할 수 있다.  
마치 검증 세트의 역할을 한다고 생각하면된다. 이 점수를 얻으려면 RandomForestClassifier 클래스의 oob_score 매개변수를 True로 지정(기본값이 False)하면 된다. 이렇게 하면 랜덤 포레스트는 각 결정 트리의 OOB 점수를 평균하여 출력해줌. 그래서 출력하면 아래와 같다. 


In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리
랜덤 포레스트와 매우 비슷하게 동작. 기본적으로 100개의 결정 트리 훈련. 결정 트리가 제공하는 대부분의 매개변수 지원.  
또한 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용.  
랜덤 포레스트와의 차이점은 부트스트랩 샘플을 사용하지 않는다는 점. 즉각 결정 트리를 만들 때 전체 훈련 세트를 사용한다.  
대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.  
앞 절에서 DecisionTreeClassifier의 splitter 매개변수를 random으로 지정했다면 엑스트라 트리에 사용하는 결정트리를 맛본 것임.  
하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블하기 때문에 과대 적합을 막고 검증 세트의 점수를 높일 수 있다.  
ExtraTreesClassifier를 사용하고 이 모델의 교차 검증 점수를 확인해 보면 다음과 같다.

In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


결과는 랜덤 포레스트와 비슷하다. 이 예제는 특성이 많지 않아 두 모델의 차이가 크지 않다.  
보통 엑스트라 트리가 무작위성이 좀 더 크기에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 함.  
하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점이다.  
엑스트라 트리도 특성 중요도를 제공.  
결과를 살펴보면 엑스트라 트리도 결정 트리보다 당도에 대한 의존성이 적다. 

In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


회귀 버전은 ExtraTreesRegressor 클래스.

## 그레이디언트 부스팅
깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식을 앙상블.  
사이킷런의 GradientBoostingClassifier는 깊이가 3인 결정 트리 100개를 사용하는게 기본값이다.  
깊이가 얕은 결정 트리를 사용하기 때문에 과대 적합에 강하고 일반적으로 높은 일반화 성능을 가지고 있다.  
그레이디언트라는 이름을 쓰는만큼 경사 하강법을 사용하여 트리를 앙상블에 추가한다.  
분류에서는 로지스틱 손실 함수를 사용하고, 회귀에서는 평균 제곱 오차 함수를 사용한다.
그레이디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동한다.  
손실 함수에서 낮은 곳으로 천천히 조금씩 이동해야 한다. 그래서 깊이가 얕은 트리를 사용하는 것이고 또 학습률 매개변수로 속도를 조절한다.

GradientBoostingClassifier를 사용해 와인 데이터셋의 교차 검증 점수를 확인.

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


과대 적합이 되지 않았다!!  
그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대 적합에 매우 강하다.  
학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있다.  

In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


결정 트리 개수를 500개로 5배나 늘렸지만 과대 적합을 잘 억제하고 있음.  
학습률 learning_rate의 기본값은 0.1.  
그레이디언트 부스팅도 특성 중요도를 제공. 결과에서 볼 수 있듯이 그레이디언트 부스팅이 랜덤 포레스트보다 일부 특성(당도)에 더 집중함.  

In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


트리 훈련에 사용할 훈련 세트의 비율을 정하는 subsample 매개변수. 이 매개변수의 기본값은 1.0으로 전체 훈련 세트를 사용.  
하지만 1보다 작으면 훈련 세트의 일부를 사용.  
마치 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하여 진행하는 확률적 경사 하강법이나 미니배치 하강법과 비슷.  
  
일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있지만 훈련 속도가 느림. 즉 GradientBoostingClassifier에는 n_jobs 매개변수가 없음. 그레이디언트 부스팅의 회귀 버전은 GradientBoostingRegressor.

## 히스토그램 기반 부스팅
히스토그램 기반 그레이디언트 부스팅. 그레이디언트 부스팅의 속도와 성능을 개선한 것.  
정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘.  
먼저 입력 특성을 256개의 구간으로 나누기에 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.  
256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용. 따라서 입력에 누락된 특성이있어도 이를 따로 전처리할 필요 없음.  
HistGradientBoostingClassifier 클래스 사용. 기본 매개변수에서도 안정적인 성능을 얻을 수 있음.  
트리에 개수를 지정할 때 n_estimators 대신 부스팅 반복 횟수를 지정하는 max_iter를 사용.

클래스 적용 시 sklearn.experimental 패키지 아래의 enable_hist_gradient_boosting 모듈을 임포트해서 씀.  
사이킷런 1.0에서 히스토그램 기반 부스팅이 experimental 모듈 아래에서 벗어났으므로 enable_hist_gradient_boosting 모듈을 임포트할 필요가 없음. 

In [10]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대 적합을 잘 억제하면서 그레이디언트 부스팅보다 조금 더 높은 성능을 제공.  
특성 중요도도 제공하니 확인.  
- permutation_importance() 함수 사용 : 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산.  
훈련 세트뿐 아니라 테스트 세트에도 적용할 수 있고 사이킷런에서 제공하는 추정기 모델에 모두 사용할 수 있음.  
  
훈련 후 특성 중요도 계산.  
n_repeats 매개 변수는 랜덤하게 섞을 횟수를 지정. 기본값은 5.

In [11]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


permutation_importance() 함수가 반환하는 객체는 반복하여 얻은 특성 중요도, 평균, 표준 편차를 담고 있음.  
평균을 출력해보면 랜덤 포레스트와 비슷한 비율임을 알 수 있음.  
테스트 세트에서 특성 중요도 계산.

In [13]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


그레이디어트 부스팅과 비슷하게 조금 더 당도에 집중하고 있음.  
이런 분석을 통해 모델을 실전에 투입했을 때 어떤 특성에 관심을 둘지 예상할 수 있음.  
성능을 최종적으로 확인해보면 아래와 같다. 

In [14]:
hgb.score(test_input, test_target)

0.8723076923076923

테스트 세트에서는 약 87% 정확도를 얻었는데 실전에 투입하면 성능은 이보다는 조금 더 낮을 것임. 앙상블 모델은 단일 결정 모델보다 좋은 결과를 얻을 수 있음. 회귀 버전은 HistGradientBoostingRegressor 클래스에 구현되어 있음.  

사이킷런 말고도 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 여럿 있음.  
그 중 하나가 XGBoost. 사이킷런의 cross_validate() 함수와 함께 쓸 수도 있고, 다양한 부스팅 알고리즘을 지원함.  
  
그럼 XGBoost를 사용해 와인 데이터의 교차 검증 점수 확인. / 마이크로소프트에서 만든 LightGBM도 확인. 

#### XGBoost

In [15]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

ModuleNotFoundError: No module named 'xgboost'

#### LightGBM

In [16]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

ModuleNotFoundError: No module named 'lightgbm'

# 정리
결정 트리 기반의 앙상블 학습은 강력하고 뚜어난 성능을 제공한다. 사이킷런에서 제공하는 앙상블 학습 알고리즘 중 랜덤 포레스트, 엑스트라 트리, 그레이디언트 부스팅, 히스토그램 기반 그레이디언트 부스팅을 다룸.  
랜덤 포레스트는 결정 트리를 훈련하기 위해 부트스트랩 샘플을 만들고 전체 특성 중 일부를 랜덤하게 선택하여 결정 트리를 만듬.  
엑스트라 트리는 랜덤 포레스트와 매우 비슷하지만 부트스트랩 샘플을 사용하지 않고 노드를 분할할 때 최선이 아니라 랜덤하게 분할. 랜덤 포레스트보다 훈련 속도가 빠르지만 보통 더 많은 트리가 필요함.  
그레이디언트 부스팅은 깊이가 얕은 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법. 성능이 뛰어나지만 병렬로 훈련할 수 없기 때문에 랜덤 포레스트나 엑스트라 트리보다 훈련 속도가 조금 느림. 그레이디언트 부스팅에서 학습률 매개변수를 조정하여 모델의 복잡도를 제어할 수 있음. 학습률 매개변수가 크면 복잡하고 훈련 세트에 과대적합된 모델을 얻을 수 있음.  
히스토그램 기반 그레이디언트 부스팅 알고리즘은 가장 뛰어나다고 평가 받는데 훈련 데이터를 256개 구간으로 변환하여 사용하기에 노드 분할 속도가 매우 빠르다. 코랩에는 XGBBoost와 LightGBM이 이미 설치되어 있어 바로 시험해볼 수 있음.  